SLP Dataset Exploration
---
This notebook explores the new public slp dataset. Towards the end, some planning takes place for training. 

In [3]:
import pandas as pd
from pathlib import Path
import numpy as np
import melee
from tqdm import tqdm
from multiprocessing import Pool
from slippi.parse import parse, ParseEvent
from slippi import Game
from tabulate import tabulate
from src.dataset.tools.melee_dataset import MeleeDataset


In [3]:
dataset_path = Path("../slp-dataset/")
training_path = dataset_path / "training_data"
training_files = list(training_path.glob("*.slp"))
print(f"{len(training_files)} files in dataset")


95102 files in dataset


In [21]:
# Create a random index to view files
random_idx = np.random.choice(len(training_files), size=10)

In [24]:
for i in random_idx:
    print(training_files[i].stem + training_files[i].suffix)
    game = Game(str(training_files[i]))
    ports = []
    players = []    
    for i, player in enumerate(game.start.players):
        if player is not None:    
            players.append(player)
            ports.append(i)

    p1 = players[0].character.name
    p1_end_stocks = game.frames[len(game.frames)-1].ports[ports[0]].leader.post.stocks
    p2_end_stocks = game.frames[len(game.frames)-1].ports[ports[1]].leader.post.stocks
    p2 = players[1].character.name    
    stage = game.start.stage.name
    print(f"{p1} {p1_end_stocks} {p2} {p2_end_stocks} {stage}")
        
    # print(game.frames[len(game.frames)-1])
    

Game_20191013T201756.slp
FALCO 3 FALCO 0 FOUNTAIN_OF_DREAMS
19_17_24 [LEAN] Fox + [YUNG] Falco (YS).slp
FOX 2 FALCO 0 YOSHIS_STORY
Game_20190824T090751.slp
FOX 4 YOSHI 4 BATTLEFIELD
Fox vs Puff (ACAB) [DL] Game_20181024T213120.slp
FOX 0 JIGGLYPUFF 2 DREAM_LAND_N64
Game_20190323T123710.slp
PEACH 2 FOX 1 BATTLEFIELD
Game_20190823T143225.slp
FOX 0 MARTH 2 POKEMON_STADIUM
Game_20190323T201724.slp
MARTH 0 FOX 2 FOUNTAIN_OF_DREAMS
Game_19490902T130404.slp
FOX 2 FALCO 2 FINAL_DESTINATION
Game_20190614T220025.slp
FOX 4 FOX 0 YOSHIS_STORY
Game_20190420T215103.slp
GANONDORF 1 GANONDORF 0 BATTLEFIELD


In [26]:
# Create an index for this dataset to easily filter games by characters
def work(path):    
    try:    
        game = Game(str(path))    
        ports = []
        players = []    
        for i, player in enumerate(game.start.players):
            if player is not None:    
                players.append(player)
                ports.append(i)

        p1 = players[0].character.name
        p2 = players[1].character.name
        
        # We only care about games that have a winner
        p1_end_stocks = game.frames[len(game.frames)-1].ports[ports[0]].leader.post.stocks
        p2_end_stocks = game.frames[len(game.frames)-1].ports[ports[1]].leader.post.stocks

        stage = game.start.stage.name
        return [path.absolute(), p1, p1_end_stocks, p2, p2_end_stocks, stage]    
        
    except:
        return [path, "err", "err", "err", "err", "err"]  # some files seem corrupted

dataset = pd.DataFrame(columns=["path", "P1", "P1_END_STOCKS", "P2", "P2_END_STOCKS" "stage"])    

pool = Pool(8)    
for idx, output in enumerate(tqdm(pool.imap_unordered(work, training_files), total=len(training_files))):
    dataset.loc[idx] = output

# There is certainly a faster way to do this using ParseEvent (probably)

100%|██████████| 95102/95102 [1:26:45<00:00, 18.27it/s]


In [27]:
# Save to file
dataset_no_err = dataset[dataset.P1 != "err"]
print(f"{len(dataset) - len(dataset_no_err)} bad examples (?)")
dataset_no_err.to_csv(dataset_path / "dataset.csv")

32 bad examples (?)


In [6]:
# Generate labels and save
df = pd.read_csv("../slp-dataset/dataset.csv")
filtered = df.copy()
label = []
for i in range(len(df)):
    row = df.iloc[i]
    if ((row['P1_END_STOCKS'] == 0 and row['P2_END_STOCKS'] != 0) 
        or (row['P1_END_STOCKS'] != 0 and row['P2_END_STOCKS'] == 0)):
        if row['P1_END_STOCKS'] == 0:
            label.append(0)  # lose
        else:
            label.append(1)  # win 
    else:
        # game incomplete
        label.append(-1)
    
filtered["label"] = label
filtered = filtered[filtered["label"] != -1]
print(len(filtered))
filtered.to_csv(dataset_path / "training.csv")      
print(filtered.iloc[0])

86847
Unnamed: 0                                                       0
path             ../slp-dataset/training_data/19_43_23 Marth + ...
P1                                                           MARTH
P1_END_STOCKS                                                    3
P2                                                           FALCO
P2_END_STOCKS                                                    0
stage                                                  BATTLEFIELD
label                                                            1
Name: 0, dtype: object


In [4]:
dataset = pd.read_csv("../slp-dataset/dataset.csv")
data = {}
for character in dataset.P1.unique():
    data[character] = (len(dataset[dataset.P1 == character]) + 
                       len(dataset[dataset.P2 == character]))/len(dataset)

headers = ['Character', '%']
data_t = sorted([(k,v) for k,v in data.items()], key=lambda x:x[1], reverse=True)
print(tabulate(data_t, headers=headers))

Character                %
--------------  ----------
FOX             0.557705
FALCO           0.380046
MARTH           0.29737
CAPTAIN_FALCON  0.251846
SHEIK           0.132902
PEACH           0.0743031
JIGGLYPUFF      0.0563269
SAMUS           0.0384033
GANONDORF       0.0299989
ICE_CLIMBERS    0.0278742
LUIGI           0.0257494
PIKACHU         0.0206479
YOSHI           0.0171873
DR_MARIO        0.013853
ZELDA           0.0129694
DONKEY_KONG     0.01217
LINK            0.00908804
MARIO           0.00877248
GAME_AND_WATCH  0.00677396
ROY             0.00479647
YOUNG_LINK      0.00465972
KIRBY           0.00440728
NESS            0.00374461
MEWTWO          0.00359735
BOWSER          0.00288209
PICHU           0.0019249


In [5]:
data = {}
for stage in dataset.stage.unique():
    data[stage] = len(dataset[dataset.stage == stage])/len(dataset)
    
headers = ['Stage', '%']
data_t = sorted([(k,v) for k,v in data.items()], key=lambda x:x[1], reverse=True)
print(tabulate(data_t, headers=headers))

Stage                             %
----------------------  -----------
BATTLEFIELD             0.239413
YOSHIS_STORY            0.176417
DREAM_LAND_N64          0.147702
POKEMON_STADIUM         0.146787
FINAL_DESTINATION       0.146629
FOUNTAIN_OF_DREAMS      0.138677
KONGO_JUNGLE            0.000925634
CORNERIA                0.000273483
RAINBOW_CRUISE          0.000231408
POKE_FLOATS             0.00022089
FLAT_ZONE               0.00022089
BRINSTAR_DEPTHS         0.000210371
MUTE_CITY               0.000189334
HYRULE_TEMPLE           0.000178816
JUNGLE_JAPES            0.000178816
BIG_BLUE                0.000168297
GREEN_GREENS            0.000168297
FOURSIDE                0.000157778
BRINSTAR                0.000157778
KONGO_JUNGLE_N64        0.00014726
MUSHROOM_KINGDOM_II     0.00014726
PRINCESS_PEACHS_CASTLE  0.000136741
GREAT_BAY               0.000136741
YOSHIS_ISLAND           0.000126223
ONETT                   0.000115704
YOSHIS_ISLAND_N64       9.46671e-05
VENOM         

Next steps: build a training dataset. There's a lot of potential options for how a dataset should be setup for this task. Some ideas:
* Fox appears in >50% of games, so a training dataset should focus on fox. 
* Filter out all the non-tournament stages. With enough games it would probably be best to focus on battlefield, but since games are limited perhaps it is best to take more stages for now.
* Dataset creation ideas:
    * Only look at each frame individually. If we know the frame leads to a certain outcome (win/loss) then we show advantage based on just this data. Ideally the sum of actions taken at each frame leads to a winning condition. 
        * Randomly sample each game for frames
        * Extract input features from sample (p1_x, p1_y, p1_damage, p1_character, p2_x, p2_y, ..., stage)
        * future: Compute the derivative of damage, stocks, as an additional feature, e.g., the rate at which P1 takes damage. Lots of processing.
    * Look at a series of contiguous frames. Same as above, but sampling for a group of N frames. Using the right model, it should better capture data about how actions chain together that lead to an advantage state. 

In [9]:
training_files = pd.read_csv("../slp-dataset/training.csv")
print(dataset.iloc[2])

Unnamed: 0                                                       2
path             ../slp-dataset/training_data/Game_20200428T212...
P1                                                             FOX
P1_END_STOCKS                                                    3
P2                                                           FALCO
P2_END_STOCKS                                                    0
stage                                                  BATTLEFIELD
Name: 2, dtype: object


In [28]:
# need to convert this to an input tensor. The 'path' variable is probably all 
# that is needed to create this tensor, whereas the other colummns can be used
# to filter games more easily. This cell will filter the training files by 
# all games FOX v (FOX, Marth, Falco, Falcon, Peach) on tournament stages
# --- don't use stages or characters that change (pokemon stadium, sheik)
dataset_filtered = training_files[(training_files.P1 == "FOX") & 
                     ((training_files.P2 == "MARTH") |
                      (training_files.P2 == "CAPTAIN_FALCON") |
                      (training_files.P2 == "FALCO") |
                      (training_files.P2 == "PEACH")) & 
                     ((training_files.stage == "BATTLEFIELD") |
                      (training_files.stage == "FINAL_DESTINATION") |
                      (training_files.stage == "YOSHIS_STORY") |
                      (training_files.stage == "DREAM_LAND_N64") |
                      (training_files.stage == "FOUNTAIN_OF_DREAMS"))
                    ]
print(f"Files for use: {len(dataset_filtered)}")

Files for use: 10145


With 60 fps gameplay and 8000-20000 frames per game, this should provide a nice amount of training data. This cell will create the input tensors ahead of time and save them along with an index containing a reference to all files we've created so far. 

The cell below iterates over the games filtered above and goes frame by frame. From each frame, extract the following:
* P1 location  (x1, y1)
* P1 damage    (D1)
* P2 location  (x2, y2)
* P2 damage    (D2)

* P1 Character (c1)
* P2 Character (c2)
* Stage        (s)

This is a 9 element tensor

Some future features that could be of use:
* player velocity  (dx, dy) 
* player average damage/sec (dD)

The other major element of dataset creation is the label. 

In [21]:
good_indices = []
labels = []
for i in range(len(dataset_filtered)):
    path = dataset_filtered.iloc[i]["path"]
    game = Game(path)
    current_stocks = [4, 4]
    for frame in game.frames:
        print(frame)
        break
    break

Frame(
    end=None,
    index=-123,
    items=( ),
    ports=(
        Port(
            follower=None,
            leader=Data(
                post=Post(
                    airborne=True,
                    character=1:FOX,
                    combo_count=0,
                    damage=0.00,
                    direction=1:RIGHT,
                    flags=0b100000000000000000000000000000000000000:DEAD,
                    ground=None,
                    hit_stun=0.00,
                    jumps=1,
                    l_cancel=None,
                    last_attack_landed=None,
                    last_hit_by=None,
                    position=(-38.80, 35.20),
                    shield=60.00,
                    state=322:ENTRY,
                    state_age=-1.00,
                    stocks=4),
                pre=Pre(
                    buttons=Buttons(
                        logical=0b0:NONE,
                        physical=0b0:NONE),
                    cstick=(0.00, 0.00),
 